In [2]:
from keras.utils import np_utils
from keras.models import load_model
import numpy as np
import scipy.misc
import os,glob,cv2,shutil,argparse,matplotlib
import tensorflow as tf
from PIL import Image

In [3]:
TFLITE_MODEL="/home/naresh/Tarento/Build_Project/gujrat_sever/trained_resnet_model_v2_10.tflite"

tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)
tflite_interpreter.allocate_tensors()

input_index = tflite_interpreter.get_input_details()
output_index = tflite_interpreter.get_output_details()


print("== Input details ==")
print("shape:", input_index[0]['shape'])
print("type:", input_index[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_index[0]['shape'])
print("type:", output_index[0]['dtype'])

== Input details ==
('shape:', array([ 1, 28, 28,  1], dtype=int32))
('type:', <type 'numpy.float32'>)

== Output details ==
('shape:', array([ 1, 11], dtype=int32))
('type:', <type 'numpy.float32'>)


In [10]:
model_path = "/home/naresh/Tarento/Build_Project/gujrat_sever/hw_recognition_api/models/trained_resnet_model_v2_10.h5"

#model = tf.keras.models.load_model("/home/naresh/Tarento/Build_Project/gujrat_sever/hw_recognition_api/models/trained_resnet_model_v2_10.h5")


# benchmarking on digit level

In [3]:
def pred_using_h5_digit(model,path):
    result = {}
    for img1 in sorted(glob.glob(path)):
        img=cv2.imread(img1)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img.astype('float32') / 255.
        img=np.reshape(img,(1,28,28,1))
        res=model.predict(img)
        pred=res[0].argmax(axis=0)
        ground = int(img1.split('/')[8])
        name = img1.split('/')[9]
        if pred!=ground:
            result[name]=pred

    return result

In [24]:
path = "/home/naresh/Tarento/Build_Project/handwritten_digit_recognition/testing_data/test/*/*.jpg"
result = pred_using_h5_digit(model,path)

# benchmarking for roll number


In [7]:
def pred_using_h5(model,path):
    result = {}
    for roll in glob.glob(path):
        ground_roll = str(roll.split('/')[8])
        pred_roll = ""
        int_med_res = {}
        for img1 in sorted(glob.glob(path.split('*')[0]+ground_roll+'/*.jpg')):
            img=cv2.imread(img1)
            img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = img.astype('float32') / 255.
            img=np.reshape(img,(1,28,28,1))
            res=model.predict(img)
            pred=res[0].argmax(axis=0)
            pred_roll=pred_roll+str(pred)
            name = img1.split('/')[9]
            int_med_res[name] = pred
        if ground_roll==pred_roll:
            result[ground_roll]=1
        else:
            print(ground_roll,  int_med_res)
            result[ground_roll]=0

    return result

In [4]:
path = "/home/naresh/Tarento/Build_Project/handwritten_digit_recognition/testing_data/site-data/*"
#result = pred_using_h5(model,path)

In [9]:
from collections import Counter
res = Counter(result.values())
res


Counter({1: 2108, 0: 479})

# test model using tflite model

In [68]:
test_img_dir = "/home/naresh/Tarento/Build_Project/gujrat_sever/test/9/*.jpg"

def test_tflite(img_path):
    result = {}
    wrong_count=0
    for img1 in sorted(glob.glob(img_path)):
        img=cv2.imread(img1)
        img=cv2.resize(img,(28,28))
        img= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.array(img,dtype=np.float32) / 255.0
        img=np.reshape(img,(1,28,28,1))
        tflite_interpreter.set_tensor(input_index[0]['index'],img)
        tflite_interpreter.invoke()
        predictions = tflite_interpreter.get_tensor(output_index[0]['index'])
        pred=predictions[0].argmax(axis=0)
        label = img1.split("/8/")[1]   ############## to get label of the image
        if int(label)!=pred:
            wrong_count+=1
            result[label]=pred
    accuracy = (len(glob.glob(img_path))-wrong_count)/len(glob.glob(img_path))
    return result,accuracy
    

In [69]:
result,accuracy = test_tflite(img_path)

69

# Data Augmentation for visualization purpose

In [56]:
#augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
image_dir = '/home/naresh/credentials/test_data2/*.jpg'
save_dir  = '/home/naresh/credentials/test_data_augmented/'
datagen = ImageDataGenerator(
        rotation_range=30,
        zoom_range=5,
        width_shift_range=0.5,
        height_shift_range=0.5,
        shear_range=0.5,
        fill_mode='nearest')
for i in sorted(glob.glob(image_dir)):
    img=load_img(i)
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # 
    j = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=save_dir, save_format='jpg'):
        j += 1
        if j > 3:
            break  # otherwise the generator would loop indefinitely